In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from itertools import chain
from nltk.corpus import wordnet
import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bigdata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./scenarii.csv", encoding="ISO-8859-1", sep=";", header=0)

,texte,tag
0,4.\tVersion Desktop - liste des cas d'utilisat...,texte
1,4.1.1\tCU001-001 Afficher le header,usecase
2,Acteur : Internaute,acteur
3,Cas dutilisation permettant à un internaute ...,texte
4,\tNotes :\nLe système affiche les éléments su...,texte


In [3]:
#Méthode qui vérifie qu'une chaine commence par un entier
def begin_num(texte):
    lis = ['0','1','2','3','4','5','6','7','8','9']
    trouvé = 0
    letter = texte[0]
    for i in lis:
        if letter == i:
            trouvé = 1
    return trouvé

In [6]:
from scipy import *

#Recupération des index des cas d'utilisation, action, Notes et Règles de gestion

i=0
useCaseTab=0
actionNoteTab=0
for x in df['tag']:
    str = df.texte[i]
    if x == 'usecase':
        a=i
        useCaseTab=append(useCaseTab,a)
    elif x == 'action' or (x == 'texte' and (not begin_num(str))) or x == 'gestion':
        b=i
        actionNoteTab=append(actionNoteTab,b)
   
    i=i+1

[   0    1    6   12   16   20   26   32   38   41   45   50   56   60
   66   70   76   80   85   93   98  104  110  120  124  129  136  141
  144  148  152  158  165  183  191  196  202  208  215  221  226  231
  236  244  252  259  267  272  278  280  282  288  293  296  301  306
  311  316  321  326  332  339  350  354  358  360  363  367  371  373
  378  386  393  402  410  415  420  426  432  438  444  450  458  466
  471  479  483  488  496  501  509  515  519  525  533  538  548  553
  563  568  574  585  596  603  613  618  623  633  635  637  645  652
  657  659  670  675  680  687  691  696  702  704  709  711  713  720
  731  735  740  743  747  752  756  758  762  769  774  776  784  793
  798  804  809  812  815  821  824  831  839  852  856  861  867  872
  879  884  890  896  905  910  915  923  934  938  943  948  965  969
  974  979  987  995 1000 1004 1011 1020 1052 1056 1060 1065 1071 1079
 1092 1099 1104 1110 1117 1126 1131 1137 1143 1202 1208 1305 1309 1326
 1341 

## Récuperation des uses cases

In [9]:
##### Récupération des cas d'utilisation
my_list_len = len(useCaseTab)
matriceCouverture=[]
for i in range(1, my_list_len):
    index = useCaseTab[i]
    matriceCouvertureLigne= df["texte"][index]
    if matriceCouvertureLigne not in matriceCouverture:
        matriceCouverture = append(matriceCouverture, matriceCouvertureLigne)

## Récuperation des exigences fonctionnelles

In [19]:
#Méthode qui vérifie qu'une chaine contient l'une des sous chaines de la liste
def is_in(texte, liste):
    trouvé = 0
    for word in liste:
        if word.lower() in texte.lower():
            trouvé = 1
        # Vérification des synonymes
        synonyms = wordnet.synsets(word, lang=u'fra')
        l=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
        for word in l:
            if word.lower() in texte.lower():
                trouvé = 1
    return trouvé

In [22]:
#Liste des mots décrivant des exigences metiers
liste = ['affiche le', 'affiché', 'ouvrir', 'mettre en surbrillance ', 'fenêtre s’ouvre ', 'cliquer', 'sélectionner', 'clic', 'utilisateur peut', 'contributeur peut', 'peut créer', 'système redirige', 'système affiche', 'se place', 'est composé', 'cliquable', 'sélectionne', 'a sélectionné', 'acceder', 'permet à', 'rediriger','permettre']

In [43]:
#Récuperation des exigences fonctionnelles
c = "\n"  #séparateur de paragraphes
s1 = "suivants :"
s3 = "suivante :"
s2 = "suivant :"
s4 = "suivants"
s5 = "affiche :"
s6 = "affiche"
s7 = "suivant"
s8 = "suivante"
vec = np.zeros((len(useCaseTab),), dtype=int)
my_list_len_actionTab = len(actionNoteTab)
matriceCouverture1=[]
matriceCouverture2=[]
i = 2
k = 0
indexi = useCaseTab[i]
for j in range(1, my_list_len_actionTab):
    indexj = actionNoteTab[j]
    if (indexj > indexi and i < len(matriceCouverture)):
        i = i+1
        k = k+1
        indexi = useCaseTab[i]
    if (indexj>useCaseTab[len(matriceCouverture)]) and (k < len(matriceCouverture)-1):
        k = k+1
    chaine = df["texte"][indexj]
    for x in chaine.split(c):
        val = is_in(x, liste)
        if val == 1 and (not (x.endswith(s1) or x.endswith(s3) or x.endswith(s3) or x.endswith(s4) or x.endswith(s5) or x.endswith(s6) or x.endswith(s7) or x.endswith(s8))):
            if x not in matriceCouverture1:
                matriceCouverture1 = append(matriceCouverture1, x)
            matriceCouverture2 = append(matriceCouverture2, x)
            vec[k] = vec[k]+1
    #if vec[k]==0:
        

In [54]:
l = matriceCouverture.transpose()
df_usecase = pd.DataFrame(l)
#df_usecase.head(10)

,0
0,4.1.1\tCU001-001 Afficher le header
1,4.1.2\tCU001-002 Afficher le bouton de langue
2,4.1.3\tCU001-0021 Afficher la page Langue
3,4.1.4\tCU001-0022 Contribuer la page langue
4,4.1.5\tCU001-003 Afficher le fil dariane
5,4.1.6\tCU001-004 Afficher le bouton de partage
6,4.1.7\tCU001-005 Afficher le menu de navigation
7,4.1.8\tCU001-006 Modifier le menu de navigation
8,4.1.9\tCU001-007 Afficher du contenu dans une ...
9,4.1.10\tCU001-008 Contribuer du contenu dans u...


In [55]:
l1 =  [matriceCouverture2]
df_EM = pd.DataFrame(l1)
#df_EM

,0,1,2,3,4,5,6,7,8,9,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,"Au clic sur la langue sélectionnée, le système...","Au clic sur la deuxième ou troisième langue, l...","Si la page est traduite, celle-ci est affichée.",\tle système redirige vers la page parente tr...,\tSi la page parente de niveau N-1 nest pas ...,\tSi la page parente de niveau N-2 nest pas ...,...,Au clic sur le bouton Partager de Twitter. Les...,Libellé Champ de partage twitter + Titre + ...,Si aucune valeur nest saisie en back-office d...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...","\tPar défaut, le système affiche les plus réc...",Si lutilisateur a sélectionné une langue angl...,Le système affiche la liste des parcours


## Construction de la matrice

In [56]:
l = matriceCouverture.transpose()
a1 = len(l)
a2 = len(matriceCouverture1)
a3=a2-1
A = np.empty((a1,a2), dtype='<U200')

mc = pd.DataFrame(A, index = l, columns = matriceCouverture1)

#mc

,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Au clic sur la deuxième ou troisième langue, le système réaffiche les boutons réseaux sociaux et le bouton + est caché","Si la page est traduite, celle-ci est affichée.", le système redirige vers la page parente traduite de niveau N-1," Si la page parente de niveau N-1 nest pas traduite, le système redirige vers la page parente traduite de niveau N-2"," Si la page parente de niveau N-2 nest pas traduite, le système redirige vers la page daccueil",...,"Au clic sur le titre dun élément, celui-ci saffiche dans une nouvelle page",Lutilisateur visualise ce bloc après avoir cliqué sur le bouton Billeterie de la télécommande,"Au clic sur le numéro de téléphone, le numéro est pré-composé sur le téléphone de lutilisateur",Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.1.1\tCU001-001 Afficher le header,,,,,,,,,,,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,,,,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,
4.1.10\tCU001-008 Contribuer du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


## Insertion des liens directs

In [60]:
id = 0
som = 0
for index_cu in range(0, len(vec)-1):
    nb_em = vec[index_cu]
    som = som + nb_em
    if nb_em > 0:
        for index_em in range(id, som):
            uc = df_usecase[0][index_cu]
            em = df_EM[index_em]
            mc.at[uc,em] = 'X'
        id = id + nb_em

In [62]:
#mc

,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Au clic sur la deuxième ou troisième langue, le système réaffiche les boutons réseaux sociaux et le bouton + est caché","Si la page est traduite, celle-ci est affichée.", le système redirige vers la page parente traduite de niveau N-1," Si la page parente de niveau N-1 nest pas traduite, le système redirige vers la page parente traduite de niveau N-2"," Si la page parente de niveau N-2 nest pas traduite, le système redirige vers la page daccueil",...,"Au clic sur le titre dun élément, celui-ci saffiche dans une nouvelle page",Lutilisateur visualise ce bloc après avoir cliqué sur le bouton Billeterie de la télécommande,"Au clic sur le numéro de téléphone, le numéro est pré-composé sur le téléphone de lutilisateur",Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.1.1\tCU001-001 Afficher le header,X,X,X,X,,,,,,,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,X,X,X,X,X,X,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,
4.1.10\tCU001-008 Contribuer du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


# DETERMINATION DES LIENS INDIRECTS A PARTIR DU KNN

## Construction du modèle KNN

In [63]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [64]:
#Mis en forme des données (Exigences Fonctionnelles) pour la construction du modèle
mt = matriceCouverture1.transpose()
data = pd.DataFrame(mt)
#data.head(10)

,0
0,"sur le logo depuis une page intérieure, le sys..."
1,"Sur le logo depuis la page daccueil, le systè..."
2,Si lutilisateur clique sur le bouton daccess...
3,Un clic sur un bouton de réseaux sociaux perme...
4,"Au clic sur la langue sélectionnée, le système..."
5,"Au clic sur la deuxième ou troisième langue, l..."
6,"Si la page est traduite, celle-ci est affichée."
7,\tle système redirige vers la page parente tr...
8,\tSi la page parente de niveau N-1 nest pas ...
9,\tSi la page parente de niveau N-2 nest pas ...


In [65]:
data.columns = ['texte']
#data.head(10)

,texte
0,"sur le logo depuis une page intérieure, le sys..."
1,"Sur le logo depuis la page daccueil, le systè..."
2,Si lutilisateur clique sur le bouton daccess...
3,Un clic sur un bouton de réseaux sociaux perme...
4,"Au clic sur la langue sélectionnée, le système..."
5,"Au clic sur la deuxième ou troisième langue, l..."
6,"Si la page est traduite, celle-ci est affichée."
7,\tle système redirige vers la page parente tr...
8,\tSi la page parente de niveau N-1 nest pas ...
9,\tSi la page parente de niveau N-2 nest pas ...


In [66]:
vec = TfidfVectorizer(max_features=500, stop_words=stopwords.words('french'))
vec.fit(data['texte'].values)
features = vec.transform(data['texte'].values)

In [69]:
#La Métrique cosine est généralement utilisé pour la similitude de texte
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
#enregistrer le modèle
from sklearn.externals import joblib
filename = 'knn.pkl'
joblib.dump(knn, filename)

In [70]:
#Observation des 10 voisins les plus proches au 1er texte ainsi que les distances qui les séparent
#knn.kneighbors(features[0:1], return_distance=True)

(array([[0.        , 0.43953408, 0.50449554, 0.52792078, 0.59270585,
         0.65035979, 0.66240437, 0.66710658, 0.71828407, 0.7216312 ]]),
 array([[  0,   1, 715, 893,  62,  95,   9,  96,  60, 113]], dtype=int64))

### Affichons les exigences metiers les plus proches de 20 premiers use case


for usecase in df_usecase[0][:20]:
    print("Use Case = ", usecase, "\n")
    input_features = vec.transform([usecase])
    D, N = knn.kneighbors(input_features, n_neighbors=10, return_distance=True)
    for input_text, distances, neighbors in zip(usecase, D, N):
        #print("Input text = ", input_text[:200], "\n")
        for dist, neighbor_idx in zip(distances, neighbors):
            if dist < 0.5 :
                print("Distance = ", dist, "Neighbor idx = ", neighbor_idx)
                print(data['texte'].values[neighbor_idx][:])
                print("-"*200)
    print("="*200)
    print('\n\n')

## INSERTION DES LIENS INDIRECT

In [73]:

for usecase in df_usecase[0]:
    #print("Input text = ", usecase, "\n")
    input_features = vec.transform([usecase])
    D, N = knn.kneighbors(input_features, n_neighbors=10, return_distance=True)
    for input_text, distances, neighbors in zip(usecase, D, N):
        #print("Input text = ", input_text[:200], "\n")
        for dist, neighbor_idx in zip(distances, neighbors):
            exi_met = data['texte'].values[neighbor_idx]
            if dist < 0.7 :
                if mc[exi_met][usecase] != 'X':
                    mc.ix[usecase.strip(),exi_met.strip()] = 'O'

In [74]:
#mc

,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Au clic sur la deuxième ou troisième langue, le système réaffiche les boutons réseaux sociaux et le bouton + est caché","Si la page est traduite, celle-ci est affichée.", le système redirige vers la page parente traduite de niveau N-1," Si la page parente de niveau N-1 nest pas traduite, le système redirige vers la page parente traduite de niveau N-2"," Si la page parente de niveau N-2 nest pas traduite, le système redirige vers la page daccueil",...,"Au clic sur le titre dun élément, celui-ci saffiche dans une nouvelle page",Lutilisateur visualise ce bloc après avoir cliqué sur le bouton Billeterie de la télécommande,"Au clic sur le numéro de téléphone, le numéro est pré-composé sur le téléphone de lutilisateur",Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.1.1\tCU001-001 Afficher le header,X,X,X,X,,,,,,,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,X,X,X,X,X,X,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,
4.1.10\tCU001-008 Contribuer du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


In [75]:
#df.to_csv (r'Path où vous souhaitez stocker le fichier CSV exporté \ File Name.csv ')

In [76]:
mc.to_excel("C:/Users/bigdata/Documents/stage Altran/code/clustering/Matric_Couverture3.xlsx") 